## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from sklearn                 import datasets, metrics, linear_model
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics         import mean_squared_error, r2_score, accuracy_score
from sklearn.tree            import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model    import LogisticRegression
from sklearn.ensemble        import GradientBoostingRegressor,GradientBoostingClassifier,RandomForestClassifier,RandomForestRegressor

warnings.filterwarnings('ignore')

In [24]:
breast_cancer = datasets.load_breast_cancer()

In [25]:
breast_cancer_x = pd.DataFrame(breast_cancer.data, columns = breast_cancer.feature_names)
breast_cancer_y = pd.DataFrame({'target':breast_cancer.target})

In [26]:
x_train, x_test, y_train, y_test = train_test_split(breast_cancer_x, breast_cancer_y, test_size= 0.2, random_state = 9)

In [27]:
gbc = GradientBoostingClassifier()

gbc.fit(x_train, y_train)

y_pred = gbc.predict(x_test)

print("Mean squared error: %.6f"% mean_squared_error(y_test, y_pred))
print("Accuracy: %.6f"% accuracy_score(y_test, y_pred))

Mean squared error: 0.043860
Accuracy: 0.956140


In [28]:
# 設定要訓練的超參數組合
n_estimators = [50, 100, 150]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(gbc, param_grid, scoring="accuracy", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  27 | elapsed:    2.2s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    2.8s finished


In [29]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.962637 using {'max_depth': 1, 'n_estimators': 100}


In [30]:
grid_result.best_params_

{'max_depth': 1, 'n_estimators': 100}

In [31]:
# 使用最佳參數重新建立模型
gbc_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
gbc_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = gbc_bestparam.predict(x_test)

In [32]:
print(metrics.mean_squared_error(y_test, y_pred))

0.043859649122807015
